In [ ]:
using Pkg; Pkg.activate(".")
using Plots, LinearAlgebra, LaTeXStrings
include("tools.jl")

# Introduction to the FFT

### Trigonometric Interpolation

Trigonometric polynomials are functions of the form
$$
t_N(x) = \sum_{k = -N}^N c_k e^{i k x}
$$
$N$ is called the degree of $t_N$. The space of all trigonometric polynomials of degree $N$ is denoted by 
$$
\mathcal{T}_N := {\rm span}\big\{ x \mapsto \exp(i k x ) \,|\, k  = -N, \dots, N \big\}.
$$



A general degree $N$ trigonometric polynomial, 
$$
   t_N(x) = \sum_{k = -N}^N c_k e^{i k x}
$$
has $2N+1$ parameters $c_{-N}, \dots, c_N$. How should we determine these? It would be ideal if we can prescribe exactly $2N+1$ conditions. A natural and general approach is *interpolation*: given a target function $f \in C_{\rm per}$ we demand that 
$$ 
	t_N(x_j) = f(x_j), \qquad j = 0, \dots, 2N
$$
where $x_j$ are called the interpolation nodes. 

How should they be chosen? It turns out  that equi-spaced nodes work very well. An intuitive justification for this choice is that in a periodic domain, all parts of the domain are "equal" and should be treated the same. By contrast in a finite interval one should *not* use equispaced nodes, but rather cluster them at the boundary (cf. Chebyshev interpolation [Trefethen, Ch. 4]). 
Because of the periodic boundary condition the nodes $x = 0, x = 2\pi$ are "equivalent", in the sense that $f(0) = f(2\pi)$, which is clearly a bad idea! A possibly way forward is to use the nodes $\frac{2\pi j}{2N+1}, j= 1, \dots, 2N+1$. For algorithmic ideas it turns out that a much more convenient decision is the following: 

Start with the grid $x_j = \pi j/N$, $j \in \mathbb{Z}$. This only gives $2N$ independent interpolation conditions, e.g. for $j = 0, \dots, 2N-1$. The interpolation operator has an at least 1-dimensional kernel. Conveniently we can exactly characterize this kernel due to the fact that 
$$
e^{i N x_j} = e^{-i N x_j},  \qquad \forall j \in \mathbb{Z}. 
$$
Namely, 
$$
   e^{i N x_j} = e^{i N j \pi/N}  = e^{i\pi j} 
  = (-1)^j = (-1)^{-j} = e^{-i \pi j} = e^{- i N \pi j/N} = e^{- i N x_j}.
$$
This suggests that we replace those two basis functions by their mean,
$$
	\frac{e^{i N x} + e^{-i N x}}{2} = \cos(N x),
$$

Effectively it means that we've replaced the space $\mathcal{T}_N$ of all trigonometric polynomials of degree $N$ with the sligtly smaller space 
$$
	\mathcal{T}_N' := {\rm span} \Big( \mathcal{T}_{N-1} \cup \{ \cos(Nx) \} \Big)
$$

But remember that $\cos(N x_j) = e^{i N x_j} = e^{- i N x_j}$ for all $x_j$, hence we can still write the interpolation conditions as 
$$
\sum_{k = -N+1}^N \hat{F}_k e^{i \pi k j / N} = F_j, \qquad j = 0, \dots, 2N-1,
$$
where $\hat{F}_k$ are the unknown parmeters and $F_j = f(x_j)$ the nodal values. 

**IMPORTANT NOTE:** The ordering of the basis is in principle arbitrary. Here we use a convention used for fast algorithms (FFT), 
$$
	(0, 1, \dots, N, -N+1, -N+2, \dots, -1)
$$
This may look strange at first, but see more on this below!

In [ ]:
"interpolation nodes"
xgrid(N) = [ j * π / N  for j = 0:2N-1 ]

"fourier coefficient indices"
kgrid(N) = [ 0:N; -N+1:-1 ]
;

It turns out that the system matrix is orthogonal up to rescaling. 

In [ ]:
N = 3
# implement the nodal interpolation operator 
A = [ exp(im * k * x) for k in kgrid(N), x in xgrid(N) ]
# observe that A'A ~ diagonal
real.(round.(A' * A, digits=12))

In [ ]:
# confirm it is an orthogonal matrix (up to scaling)! I.e. (A'/2N) = inv(A) !!
norm(A' * A - 2*N*I)

In addition to guaranteeing that $I_N$ is well-defined we also see that the matrix $A$ we need to invert is orthogonal (up to rescaling), which makes it very easy to invert it. We just need to multiply by $A^H$, i.e. $O(N^2)$ computational cost instead of $O(N^3)$ for solving a full linear system via [Gaussian elimination](https://en.wikipedia.org/wiki/LU_decomposition).

These two operations $F \mapsto \hat{F}$ and $\hat{F} \mapsto F$ are called the discrete and inverse discrete fourier transforms (up to scaling). They can in fact be applied with $O(N \log(N))$ computational cost, using the *fast fourier transform*. We will discuss this below, but first use our naive implementation.

In [ ]:
"""
construct the coefficients of the trigonometric interpolant
"""
function triginterp(f, N)
    X = xgrid(N)
    # nodal values at interpolation nodes
    F = f.(X) 
    # system matrix
    A = [ exp(im * x * k) for k in kgrid(N), x in X ]
    # coefficients are given by F̂ = A' * F as discussed above!
    return (A' * F) / (2*N)
end 


"""
to evaluate a trigonometric polynomial just sum coefficients * basis
we the take the real part because we assume the function we are 
approximating is real.
"""
evaltrig(x, F̂) = sum( real(F̂k * exp(im * x * k))
                      for (F̂k, k) in zip(F̂, kgrid(length(F̂) ÷ 2)) )


### The Fast Fourier Transform 

Recall from above that the trigonometric interpolant $I_N f$ of a function $f$ is given by
$$
	I_N f(x) = \sum_{k = -N+1}^{N-1} \hat{F}_k e^{i k x} + \hat{F}_N \cos(N x)
$$
and the coefficients are determined by the linear system 
$$
	\sum_{k = -N+1}^N \hat{F}_k e^{i k x_j} = F_j, \qquad j = 0, \dots, 2N-1.
$$
where $F_j = f(x_j)$ and $x_j = j \pi / N$. We have moreover shown numerically (to be proven in assignment) that the system matrix is orthogonal (up to rescaling), i.e., if 
$$
	A = \big( e^{i k x_j} \big)_{k,j}
$$
then 
$$
	A A^H = 2N I
$$
In particular $A$ is invertible, i.e., the mapping $F \mapsto \hat{F}, \mathbb{C}^{2N} \to \mathbb{C}^{2N}$ is invertible. 
This mapping is called the discrete fourier transform (DFT) and its inverse is called the inverse discrete fourier transform (IDFT, $\hat{F} \mapsto F$). Both use a different scaling than we use here; specifically, the most commen definition is 
$$
\begin{aligned}
	{\rm DFT}[G]_k &= \sum_{j = 0}^{2N-1} e^{- i k j \pi / N} G_j, \\ 
	{\rm IDFT}[\hat{G}]_j &= \frac{1}{2N} \sum_{k = -N+1}^N e^{i k j \pi / N} \hat{G}_k.
\end{aligned}
$$
This means the the mappings $F \mapsto \hat{F}, \hat{F} \mapsto F$ can be written as 
$$
	\hat{F} = (2N)^{-1} \cdot {\rm DFT}[F], \qquad F = 2N \cdot {\rm IDFT}[\hat{F}]
$$


The cost of evaluating the DFT and IDFT naively is $O(N^2)$ (matrix-vector multiplication) but the special structures in the DFT make it possible to evaluate them in $O(N \log (N))$ operations. This was first observed by Gauss (1876), and much later rediscovered and popularized by [Cooley & Tukey (1965)](https://en.wikipedia.org/wiki/Cooley–Tukey_FFT_algorithm). It is generally considered one of the [most important algorithms of the 20th century](https://www.computer.org/csdl/magazine/cs/2000/01/c1022/13rRUxBJhBm). 

In Julia, the FFT is implemented in the [FFTW package](https://github.com/JuliaMath/FFTW.jl) (the Fastest Fourier Transform in the West). Before we study it, we can try it out:


In [ ]:
using FFTW

function dft(F)
    N = length(F) ÷ 2
    A = [ exp(im * k * x) for k in kgrid(N), x in xgrid(N) ]
    return (A' * F) / (2*N)
end

In [ ]:
?fft

In [ ]:
# run a random tests to confirm FFT = DFT
N = 128
F = rand(ComplexF64, N)
norm( dft(F) - fft(F) / N )

In [ ]:
N = 128
# run a random test to see how fft, ifft work
F = rand(ComplexF64, N)
norm(F - ifft(fft(F)))

In [ ]:
using BenchmarkTools

In [ ]:
NN_dft = [ 8, 16, 32, 64, 128 ] 
NN_fft = [ 8, 16, 32, 64, 128, 256, 512, 1024, 2048 ] 
FF = [ rand(ComplexF64, 2*N) for N in NN_fft ]   # random trial vectors 
times_dft = [ (@belapsed dft($(FF[n]))) for n = 1:length(NN_dft) ]
times_fft = [ (@belapsed fft($(FF[n]))) for n = 1:length(NN_fft) ];

In [ ]:
# plot(NN, times_dft, label = "DFT", lw=2, xscale = :log10, yscale=:log10, m=:o, ms=5)

plot(NN_dft, times_dft, lw=2, m=:o, label = "DFT", 
     xscale = :log10, yscale=:log10, ylims = (1e-6, 3e-3), 
    size = (400, 300), xlabel = "N", ylabel = "runtime [s]")
plot!(NN_fft, times_fft, lw=2, m=:o, label = "DFT")
plot!(NN_dft, 1e-7*NN_dft.^2, lw=1, ls=:dash, c=:black, label = L"N^2, N")
plot!(NN_fft, 3e-8*NN_fft, lw=1, ls=:dash, c=:black, label = "")